Задание 1. Реализовать tfidf датасета 

In [1]:
# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

File ‘./quora.txt’ already there; not retrieving.


In [2]:
#Установка нужных пакетов
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import pandas as pd
import umap
from nltk.tokenize import RegexpTokenizer

from matplotlib import pyplot as plt

from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from IPython.display import clear_output

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
data = list(open("./quora.txt", encoding="utf-8"))[0:2000]
data[0]

"Can I get back with my ex even though she is pregnant with another guy's baby?\n"

In [11]:
def preproc(st):
  tokenizer = RegexpTokenizer(r"\w+")

  data_tok = tokenizer.tokenize(st.lower()) 

  data_word = []

  stopw = stopwords.words('english') 

  for word in data_tok:
    if word not in stopw and len(word) > 1:
      data_word.append(word)

  ## Стемминг
  # print('stemming and lemmatize')
  ps = PorterStemmer()
  lemmatizer = WordNetLemmatizer()

  data_tok_lem_stem = []

  all_lem = [ps.stem(i) for i in data_word]
  data_tok_lem_stem = [lemmatizer.lemmatize(i) for i in all_lem]

  return data_tok_lem_stem

In [12]:
def vocab(data_tok_lem_stem):
  vocab = []
  print()
  print('Составляем словарь')

  for words in tqdm(data_tok_lem_stem):
    for word in words:
      vocab.append(word)

  vocab = list(set(vocab))

  return vocab

In [13]:
def tfidf(data_tok_lem_stem, vocab):
  d = {}
  list_name = []
  list_tf = []

  print('составляем TF')
  for word in tqdm(vocab):
    list_tf = []
    for st in data_tok_lem_stem:
      i = 0
      for lem in st:

        #Находим кол_во вхождений в предложение
        if lem == word:
          i += 1 
      if i == 0:
        list_tf.append(0)
      else:
        list_tf.append(i/len(st))
    d[word] = list_tf

  df_tf = pd.DataFrame(d).T

  dict_idf = {}

  print('составляем IDF')
  for word in tqdm(vocab):
    count_doc_in = 0
    for st in data_tok_lem_stem:
      if word in st:
        count_doc_in += 1
    idf_tok = np.log(len(data_tok_lem_stem)/count_doc_in)
    dict_idf[word] = idf_tok

  df_idf = pd.DataFrame(dict_idf, index=['idf']).T
  df_tfidf = df_tf.merge(df_idf, how = 'left', left_index=True, right_index=True)

  for column in df_tfidf.columns:
    df_tfidf[column] = df_tfidf[column] * df_tfidf['idf']
  df_tfidf.drop('idf', axis = 1, inplace = True)

  return df_tfidf


In [14]:
def cosine(vector1, vector2):
  # рассчитаем скалярное произведение векторов
  numerator = np.dot(vector1, vector2)

  # (1) рассчитаем длины (по большому счету, это теорема Пифагора)
  vector1Len = np.linalg.norm(vector1)
  vector2Len = np.linalg.norm(vector2)
  
  # (2) перемножим их
  denominator = vector1Len * vector2Len

  # посмотрим, чему равен косинус угла между векторами
  cosine = numerator/denominator
  return cosine

In [15]:
data_tok_lem_stem = []
for st in tqdm(data):
  data_tok_lem_stem.append(preproc(st))
vocab = vocab(data_tok_lem_stem)

100%|██████████| 2000/2000 [00:00<00:00, 3586.77it/s]



Составляем словарь


100%|██████████| 2000/2000 [00:00<00:00, 821929.06it/s]


In [16]:
df_tfidf = tfidf(data_tok_lem_stem, vocab)

составляем TF


100%|██████████| 3898/3898 [00:02<00:00, 1330.40it/s]


составляем IDF


100%|██████████| 3898/3898 [00:00<00:00, 4437.90it/s]


In [17]:
df_tfidf

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
addict,0.0,1.300458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
core,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
hostel,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
transform,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
npr,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
stop,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.450306,0.0,0.0,0.0
fit,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
paperwhit,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
ib,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [18]:
cos_dict = {}
all_list_df_tfidf = df_tfidf.T.index.to_list()

for vect1 in tqdm(all_list_df_tfidf):
  l_cos = []
  # for vect2 in df_tfidf.T.loc[vect1: df_tfidf.T.last_valid_index()].index.to_list():
  for vect2 in all_list_df_tfidf:
    cos = cosine(df_tfidf.T.loc[vect1].to_list(), df_tfidf.T.loc[vect2].to_list())
    l_cos.append(cos)
  cos_dict[vect1] = l_cos

  0%|          | 0/2000 [00:00<?, ?it/s]<ipython-input-14-d3ba0a69f867>:13: RuntimeWarning: invalid value encountered in double_scalars
  cosine = numerator/denominator
  0%|          | 9/2000 [00:33<2:02:05,  3.68s/it]


KeyboardInterrupt: ignored

In [ ]:
d_cos = pd.DataFrame(cos_dict, index = list(cos_dict.keys()))
d_cos

In [ ]:
for i in d_cos.index.to_list():
  d_cos.at[i,i] = 0

In [ ]:
dd = d_cos.merge(d_cos.max(axis = 1).to_frame(name="max_val"), left_index = True, right_index = True).sort_values('max_val', ascending = False)
dd['max_zavis_column'] = dd.idxmax()
print('Cвязанные предложения')
dd[['max_val','max_zavis_column']]